In [ ]:
print("Checking the kernel is working")

# Mechaphlowers : basics

## Installation 

### On jupyter server

In [ ]:
# %pip install mechaphlowers nbformat # uncomment if interested
# import mechaphlowers as mph
# mph.__version__

### On jupyterlite (pyodide)

In [ ]:
import micropip
await micropip.install(["mechaphlowers","nbformat"])
import mechaphlowers as mph
mph.__version__

## 1. Add a power line section and display it 

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from mechaphlowers import SectionDataFrame
from mechaphlowers.entities.arrays import SectionArray


In [ ]:
# load data 
data = {
    "name": ["1", "2", "three", "support 4"],
    "suspension": [False, True, True, False],
    "conductor_attachment_altitude": [50.0, 40.0, 20.0, 10.0],
    "crossarm_length": [5.0,]* 4,
    "line_angle": [0.]* 4,
    "insulator_length": [0, 4, 3.2, 0],
    "span_length": [100, 200, 300, np.nan],
}

section = SectionArray(data=pd.DataFrame(data))

# set sagging parameter and temperatur 
section.sagging_parameter = 500
section.sagging_temperature = 15

# Provide section to SectionDataFrame
frame = SectionDataFrame(section)

# Display figure
fig = go.Figure()
frame.plot.line3d(fig)
fig.show()

📌 As the line can be difficult to visualize due to its linear nature, here are two possibilities for better visualization:
- Select a part of the line

In [ ]:
fig._data = []
frame.select(between=["three", "support 4"]).plot.line3d(fig)
fig.show()

- Use a preset layout to have a more compact view with view sets to analysis. Please note that this presetting does not respect proportions.

In [ ]:
fig._data = []
frame.plot.line3d(fig, view="analysis")
fig.show()

## Issues for displaying plots ? 

📌 It can be tricky to set the good parameters to move the 3D graph as desired. We want here to help in providing some ready-to-use code.

- Set the window size for the graph with height and width parameters

In [ ]:
fig.update_layout(autosize=False, height=600, width=1000)

- inverse view

In [ ]:
fig.update_layout(
    autosize=False, height=800, width=1000, 
    scene_camera=dict(
        eye=dict(x=0, y=1, z=0)
))

- Set profil view

In [ ]:
fig.update_layout(
    autosize=False, height=800, width=1000, 
    scene_camera=dict(
        eye=dict(x=.9, y=-0, z=-0.1)
))

- Set view from above

In [ ]:
fig.update_layout(
    autosize=False, height=800, width=1500, 
    scene_camera=dict(
        eye=dict(x=0, y=.1, z=1)
))

## 2. Add a cable to the section and compute some parameters

In [ ]:
from mechaphlowers.entities.arrays import CableArray

In [ ]:
cable_data = pd.DataFrame(
		{
			"section": [345.55],
			"diameter": [22.4],
			"linear_weight": [9.55494],
			"young_modulus": [59],
			"dilatation_coefficient": [23],
			"temperature_reference": [0],
		}
	)
cable_array = CableArray(cable_data.loc[cable_data.index.repeat(4)].reset_index(drop=True))
cable_array.data

In [ ]:
frame.add_cable(cable_array)
frame.data

In [ ]:
display("Length", frame.span.L(),
"Unstressed cable length at 20°", frame.state.L_ref(20),
"Unstressed cable length at -30°", frame.state.L_ref(-30),)

## 3. Add weather conditions span by span and compute associated loads

In [ ]:
from mechaphlowers.entities.arrays import WeatherArray

weather = WeatherArray(
	pd.DataFrame(
		{
			"ice_thickness": [1, 2.1, 0.0, 5.4],
			"wind_pressure": [40.12, 0.0, 12.0, 53.0],
		}
	)
)
frame.add_weather(weather=weather)
frame.data

In [ ]:
display(
    "Loads due to wind", frame.cable_loads.wind_load,
    "Loads due to ice", frame.cable_loads.ice_load,
)

## 4. Plot line section with external loads

📌 For the moment maximum wind is applied on each span

In [ ]:
fig = go.Figure()
frame.plot.line3d(fig, view="analysis")
fig.update_layout(
    autosize=False, height=800, width=1000, 
    scene_camera=dict(
        eye=dict(x=.9, y=.1, z=-0.1)
))

🤩 Array and frame objects are dynamically linked together. Let's try to push the cable a little more !

In [ ]:
weather._data.loc[0, "wind_pressure"] = np.float64(10000.)
weather.data

In [ ]:
fig._data = []
frame.plot.line3d(fig,view="analysis")
fig.update_layout(
    autosize=False, height=800, width=1500, 
    scene_camera=dict(
        eye=dict(x=.9, y=.1, z=-0.1)
))